In [8]:
from sqlite3 import connect
from pandas import read_csv, Series, read_sql
import pandas as pd
pd.options.mode.chained_assignment = None
class IdentifiableEntity(object):
        def __init__(self, id:str):
            self.id = id
        def getId(self):
            return self.id


class Image(IdentifiableEntity):
        def __init__ (self,id):
            super().__init__(id)


class Annotation(IdentifiableEntity): 
        def __init__(self, id, motivation, body:Image, target:IdentifiableEntity):
            self.motivation = motivation
            self.body = body
            self.target = target
            super().__init__(id)
        def getBody(self):
            return Image
        def getMotivation(self):
            return self.motivation
        def getTarget(self):
            return IdentifiableEntity
        


We can consider the class Annotation as a blueprint, now we need to connect real data with the class using instances. 

- query database to extract information that I need -> output: Dataframe.

    If the information is stored in both databases we need to query both -> 2 dataframes -> merge(.concat) 2 dataframes -> 1 output Dataframe
- the output dataframe is a representation of the class -> the columns need to correspond to the attributes of the class.
- Iterate over the rows of the dataframe
- For each row, it creates a new instance of the Annotation class-> the class attributes now are replaced by the corresponding columns.
- Append python object to a list. 

In [9]:
with connect("relational.db") as con: #getAllAnnotations method
    query_1 = "SELECT annotation,target,body,motivation FROM Annotation LEFT JOIN image ON Annotation.imageId == image.imageId"
    all_annotations_query = read_sql(query_1,con)
    annotations = []
    for x, row in all_annotations_query.iterrows():
        annotation = Annotation(id=row['annotation'], motivation=row['motivation'], body=row['body'], target=row['target'])
        annotations.append(annotation)
annotations

In [10]:
with connect("relational.db") as con:
    query_2 = "SELECT body FROM Image"
    all_images_query = read_sql(query_2,con)
    images = []
    for x, row in all_images_query.iterrows():
        image = Image(id=row['body'])
        images.append(image)
images

In [11]:

with connect("relational.db") as con: 
    query_3 = "SELECT annotation,target,body,motivation FROM Annotation LEFT JOIN image ON Annotation.imageId == image.imageId WHERE body='https://dl.ficlit.unibo.it/iiif/2/45498/full/699,800/0/default.jpg'"
    annotations_with_body_query = read_sql(query_3,con)
aw = []
for x, row in annotations_with_body_query.iterrows():
    aw1= Annotation(id=row['annotation'], motivation=row['motivation'], body=row['body'], target=row['target'])
    aw.append(aw1)
aw

In [12]:
from sqlite3 import connect
from pandas import read_csv, Series, DataFrame, read_sql
import pandas as pd

with connect("relational.db") as con: 
    query_1 = "SELECT annotation,target FROM Annotation WHERE annotation='https://dl.ficlit.unibo.it/iiif/2/28429/annotation/p0001-image'"
    annotations_query = read_sql(query_1, con)
annotations_query
    # annotations_query = annotations_query["target"]
    # x = annotations_query[0]
    # print(x)


,annotation,target
0,https://dl.ficlit.unibo.it/iiif/2/28429/annota...,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p1


fare una serie di if statements per differenziare ogni tipo di input id che viene fatto passare (annotations, image, canvas, collection, manifest)
nei primi due casi:
create an auxiliary function in which the parameter is x -> used to retrieve information from graph database
extract id and type

note metodi "generic query processor" che bisogna unire:
- getAnnotationsToCollection: 
    1) input collection contenuta nel graph database 
    2) query nel graph -> da quella collection estrarre tutti i canvas relativi
    3) prendi gli id canvas come input e applica la stessa funzione con questi nuovi input per fare una query nel relational database. 
- getAnnotationsToManifest -> fare la stessa cosa. 
- getEntitybyId -> ricalcare quanto fatto nel query processor + farlo aderire alla classe EntityWithMetadata

In [16]:
from sqlite3 import connect
from pandas import read_csv, Series, DataFrame, read_sql
import pandas as pd

with connect("relational.db") as con: 
    query_1 = "SELECT DISTINCT id,creators,title FROM EntityWithMetadata WHERE id LIKE '%canvas%'"
    query = read_sql(query_1, con)

"SELECT DISTINCT id,creators,title FROM EntityWithMetadata WHERE id LIKE '%collection%'"
"SELECT DISTINCT id,creators,title FROM EntityWithMetadata WHERE id LIKE '%manifest%'"
with connect("relational.db") as con: 
    query_1 = "SELECT * FROM Annotation LEFT JOIN image ON Annotations.imageId == image.imageId WHERE target=?"
    query = read_sql(query_1, con, params=(target,))
"SELECT * FROM Annotation LEFT JOIN image ON Annotations.imageId == image.imageId WHERE target LIKE '%collection%'"
"SELECT * FROM Annotation LEFT JOIN image ON Annotations.imageId == image.imageId WHERE target LIKE '%manifest%'"
"SELECT * FROM Annotation LEFT JOIN image ON Annotations.imageId == image.imageId WHERE body=?"
query = read_sql(query_1, con, params=(body,))
"SELECT * FROM Annotation LEFT JOIN image ON Annotations.imageId == image.imageId WHERE body=? AND target=?"
annotations_with_body_and_target_query = read_sql(query_4,con,params=(bodyId, targetId))
"SELECT * FROM Annotation LEFT JOIN image ON Annotations.imageId == image.imageId WHERE target=?"
annotations_with_body_and_target_query = read_sql(query_4,con,params=(targetId,))
"SELECT DISTINCT id,creators,title FROM EntityWithMetadata WHERE id LIKE '%canvas%'"
"SELECT DISTINCT id,creators,title FROM EntityWithMetadata WHERE id LIKE '%manifest%'"
#skipped getEntitybyId
query_6 = "SELECT DISTINCT * FROM EntityWithMetadata LEFT JOIN creators_table ON EntityWithMetadata.creatorId == creators_table.creatorId WHERE creator=?"
         entities_with_creator_query = read_sql(query_6,con,params=(creator_name,)) #missing label
"SELECT DISTINCT id,creators,title FROM EntityWithMetadata WHERE id=" #missing parameter that will be passed from sparql query
query_7 = "SELECT  DISTINCT * FROM EntityWithMetadata WHERE title=?"
        entities_with_title_query = read_sql(query_7,con,params=(title,))
"SELECT body FROM  Annotation LEFT JOIN image ON Annotations.imageId == image.imageId WHERE target=?"
query = read_sql(query_1, con, params=(target,))
"SELECT DISTINCT id,creators,title FROM EntityWithMetadata WHERE id=''" #the input will be passed from the sparql query




,id,creators,title
0,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p1,,
1,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p2,,
2,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p3,,
3,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p4,,
4,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p5,,
...,...,...,...
266,https://dl.ficlit.unibo.it/iiif/2/19425/canvas...,,
267,https://dl.ficlit.unibo.it/iiif/2/19425/canvas...,,
268,https://dl.ficlit.unibo.it/iiif/2/19425/canvas...,,
269,https://dl.ficlit.unibo.it/iiif/2/19425/canvas...,,
